# Running simulations through the cloud 

This notebook is a tutorial of the API used for submitting simulations to our servers.


In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


02:39:54 PST Created task 'webAPI' with task_id                                 
             'fdve-12ff73a4-b398-4fc8-ab90-e48855af189b' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=786536;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=419746;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\taskId]8;;\]8;id=786536;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\=]8;;\]8;id=941816;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\fdve]8;;\]8;id=786536;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\-12ff73a4-b39]8;;\
             ]8;id=786536;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\8-4fc8-ab90-e48855af189b']8;;\.

Output()

02:39:56 PST status = queued

Output()

...

Output()

02:40:10 PST status = success

             View simulation result at                                          
             ]8;id=775166;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=237389;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\taskId]8;;\]8;id=775166;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\=]8;;\]8;id=138709;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\fdve]8;;\]8;id=775166;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\-12ff73a4-b39]8;;\
             ]8;id=775166;https://tidy3d.simulation.cloud/workbench?taskId=fdve-12ff73a4-b398-4fc8-ab90-e48855af189b\8-4fc8-ab90-e48855af189b']8;;\.

Output()

             loading simulation from data/sim.hdf5

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../api/_autosummary/tidy3d.web.run.html) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../api/_autosummary/tidy3d.web.start.html) and then exit the session and load the results at a later time using [web.load](../api/_autosummary/tidy3d.web.load.html).


## Job Container

The [web.Job](../api/_autosummary/tidy3d.web.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../api/_autosummary/tidy3d.Simulation.html).

We can get the cost estimate of running the task before actually running it. This prevents us from accidentally running large jobs that we set up by mistake. The estimated cost is the maximum cost corresponding to running all the time steps.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# estimate the maximum cost
estimated_cost = web.estimate_cost(job.task_id)

02:40:11 PST Created task 'job' with task_id                                    
             'fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=860710;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=28989;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\taskId]8;;\]8;id=860710;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\=]8;;\]8;id=241239;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\fdve]8;;\]8;id=860710;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\-6c1807ab-bc1]8;;\
             ]8;id=860710;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6c1807ab-bc1e-4631-9ef6-5b42351d8d90\e-4631-9ef6-5b42351d8d90']8;;\.

Output()

02:40:12 PST Maximum FlexCredit cost: 0.025. Minimum cost depends on task       
             execution details. Use 'web.real_cost(task_id)' to get the billed  
             FlexCredit cost after a simulation run.

While [Job](../api/_autosummary/tidy3d.web.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [5]:
# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")

             status = success

Output()

             loading simulation from data/sim.hdf5

Another convenient thing about [Job](../api/_autosummary/tidy3d.web.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [6]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

02:40:13 PST loading simulation from data/sim.hdf5

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../api/_autosummary/tidy3d.web.Batch.html) object.
The batch object is like a [Job](../api/_autosummary/tidy3d.web.Job.html), but stores task metadata for a series of simulations.

In [7]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


             Created task 'sim_0' with task_id                                  
             'fdve-b305a482-e635-4388-9cb0-c45909ce8782' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=484688;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=372060;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\taskId]8;;\]8;id=484688;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\=]8;;\]8;id=827712;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\fdve]8;;\]8;id=484688;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\-b305a482-e63]8;;\
             ]8;id=484688;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b305a482-e635-4388-9cb0-c45909ce8782\5-4388-9cb0-c45909ce8782']8;;\.

Output()

             Created task 'sim_1' with task_id                                  
             'fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=723062;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=775632;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\taskId]8;;\]8;id=723062;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\=]8;;\]8;id=223351;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\fdve]8;;\]8;id=723062;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\-4c6113e6-6e7]8;;\
             ]8;id=723062;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94\c-4405-a7ae-232623ecae94']8;;\.

...

02:40:15 PST Started working on Batch.

02:40:16 PST Maximum FlexCredit cost: 0.075 for the whole batch.

             Use 'Batch.real_cost()' to get the billed FlexCredit cost after the
             Batch has completed.

Output()

02:40:17 PST Batch complete.

When the batch is completed, the output is not a [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../api/_autosummary/tidy3d.web.BatchData.html).  The data within this [BatchData](../api/_autosummary/tidy3d.web.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [8]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

             loading simulation from                                            
             data/fdve-b305a482-e635-4388-9cb0-c45909ce8782.hdf5

/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


Output()

02:40:18 PST loading simulation from                                            
             data/fdve-4c6113e6-6e7c-4405-a7ae-232623ecae94.hdf5

/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


Output()

             loading simulation from                                            
             data/fdve-55e39689-a0a2-4269-b044-f7feebb8974f.hdf5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../api/_autosummary/tidy3d.web.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [9]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


02:40:19 PST Created task 'sim_0' with task_id                                  
             'fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=657779;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=525616;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\taskId]8;;\]8;id=657779;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\=]8;;\]8;id=260289;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\fdve]8;;\]8;id=657779;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\-2e62b18c-751]8;;\
             ]8;id=657779;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90\f-43ec-835c-0bf85c9e0a90']8;;\.

Output()

             Created task 'sim_1' with task_id                                  
             'fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=341513;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=651593;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\taskId]8;;\]8;id=341513;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\=]8;;\]8;id=857336;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\fdve]8;;\]8;id=341513;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\-0e1e7619-892]8;;\
             ]8;id=341513;https://tidy3d.simulation.cloud/workbench?taskId=fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975\d-488e-b1b4-3955ab8f6975']8;;\.

...

02:40:20 PST Started working on Batch.

02:40:21 PST Maximum FlexCredit cost: 0.075 for the whole batch.

             Use 'Batch.real_cost()' to get the billed FlexCredit cost after the
             Batch has completed.

Output()

02:40:22 PST Batch complete.

In [10]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

02:40:23 PST loading simulation from                                            
             ./fdve-2e62b18c-751f-43ec-835c-0bf85c9e0a90.hdf5

/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


Output()

             loading simulation from                                            
             ./fdve-0e1e7619-892d-488e-b1b4-3955ab8f6975.hdf5

/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


Output()

02:40:24 PST loading simulation from                                            
             ./fdve-d5c5085f-4fdc-421a-80ab-51ef5c77aa7c.hdf5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


/tmp/ipykernel_2106502/711851585.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sum_intensity = float(intensity.sum(("x", "y")).values)


After going through this tutorial, you have learned how to run simulations with Tidy3D via the web API. If you are new to Tidy3D or the finite-difference time-domain (FDTD) method, we highly recommend going through our [FDTD101](https://www.flexcompute.com/fdtd101/) tutorials and [example library](https://www.flexcompute.com/tidy3d/examples/) before starting your own simulation adventure. 